In [ ]:
# STEP 1: clean + install a fresh OpenCV that works with current NumPy
!pip uninstall -y opencv-python opencv-python-headless

# install latest compatible headless OpenCV (works with NumPy 2.x)
!pip install -q opencv-python-headless

# quick test
from google.colab.patches import cv2_imshow
import cv2, numpy as np

print("OpenCV version:", cv2.__version__)
print("NumPy version:", np.__version__)


In [ ]:
# REAL-TIME COLOR DETECTOR & TRACKER (PROJECT CODE) --------------------------
from IPython.display import clear_output
from google.colab import output
from google.colab.patches import cv2_imshow
import cv2, numpy as np, time
from base64 import b64decode

# JS: capture one frame from browser webcam
capture_js = """
async function captureFrame(quality=0.85, timeout_ms=5000) {
  const start = performance.now();
  let stream = null;
  try {
    stream = await navigator.mediaDevices.getUserMedia({video: true, audio: false});
  } catch(e) {
    return {error: 'getUserMedia failed: ' + e.toString()};
  }
  const video = document.createElement('video');
  video.srcObject = stream;
  video.playsInline = true;
  try { await video.play(); } catch(e) {}

  // wait until video has valid size or timeout
  while ((video.videoWidth === 0 || video.videoHeight === 0) &&
         (performance.now() - start) < timeout_ms) {
    await new Promise(r => setTimeout(r, 50));
  }

  if (video.videoWidth === 0 || video.videoHeight === 0) {
    stream.getTracks().forEach(t => t.stop());
    return {error: 'camera did not provide dimensions in time'};
  }

  const canvas = document.createElement('canvas');
  canvas.width = video.videoWidth;
  canvas.height = video.videoHeight;
  const ctx = canvas.getContext('2d');
  ctx.drawImage(video, 0, 0, canvas.width, canvas.height);

  stream.getTracks().forEach(track => track.stop());
  const dataUrl = canvas.toDataURL('image/png', quality);
  return {data: dataUrl};
}
"""

def grab_frame(quality=0.8, tries=2):
  """Capture single frame from webcam via JS and return as BGR image."""
  for _ in range(tries):
    result = output.eval_js(capture_js + f"captureFrame({quality}, 5000)")
    if not result:
      time.sleep(0.3)
      continue

    # handle dict or string
    if isinstance(result, dict) and 'error' in result:
      raise RuntimeError("JS error: " + result['error'])

    data_url = None
    if isinstance(result, dict) and 'data' in result:
      data_url = result['data']
    elif isinstance(result, str) and result.startswith("data:image"):
      data_url = result
    else:
      s = str(result)
      if "data:image" in s:
        idx = s.find("data:image")
        data_url = s[idx:]

    if not data_url:
      time.sleep(0.2)
      continue

    try:
      header, b64 = data_url.split(',', 1)
      img_bytes = b64decode(b64)
      nparr = np.frombuffer(img_bytes, np.uint8)
      img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
      return img
    except Exception as e:
      raise RuntimeError("Failed to decode frame: " + str(e))

  raise RuntimeError("Failed to capture a frame from camera.")

# HSV COLOR RANGES -----------------------------------------------------------
COLOR_RANGES = {
    "red1":  ((0, 120, 70),  (10, 255, 255)),
    "red2":  ((170, 120, 70),(180, 255, 255)),
    "green": ((35, 80, 40),  (85, 255, 255)),
    "blue":  ((90, 60, 40),  (140, 255, 255)),
    "yellow":((15, 120, 120),(35, 255, 255)),
    "orange":((10, 120, 100),(25, 255, 255)),
    "pink":  ((145, 80, 80), (170, 255, 255)),
}

def detect_and_track(frame, color_name="green", min_area=1000, debug=False):
  """Detect given color in frame and draw circle + centroid on it."""
  hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

  # handle red as 2 ranges
  if color_name == "red":
    m1 = cv2.inRange(hsv, np.array(COLOR_RANGES["red1"][0]),
                          np.array(COLOR_RANGES["red1"][1]))
    m2 = cv2.inRange(hsv, np.array(COLOR_RANGES["red2"][0]),
                          np.array(COLOR_RANGES["red2"][1]))
    mask = cv2.bitwise_or(m1, m2)
  else:
    if color_name not in COLOR_RANGES:
      raise ValueError(f"Unknown color: {color_name}")
    lower, upper = COLOR_RANGES[color_name]
    mask = cv2.inRange(hsv, np.array(lower), np.array(upper))

  # clean mask
  kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
  mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=1)
  mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=1)
  mask = cv2.GaussianBlur(mask, (7,7), 0)

  contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  annotated = frame.copy()

  if contours:
    c = max(contours, key=cv2.contourArea)
    area = cv2.contourArea(c)
    if area > min_area:
      (x,y), r = cv2.minEnclosingCircle(c)
      M = cv2.moments(c)
      if M["m00"] != 0:
        cx, cy = int(M["m10"]/M["m00"]), int(M["m01"]/M["m00"])
      else:
        cx, cy = int(x), int(y)

      cv2.circle(annotated, (int(x), int(y)), int(r), (0,255,0), 2)
      cv2.circle(annotated, (cx, cy), 5, (0,0,255), -1)
      cv2.putText(annotated, f"{color_name} area:{int(area)}", (10,30),
                  cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)

      if debug:
        mask_bgr = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
        h,w = annotated.shape[:2]
        small = cv2.resize(mask_bgr, (w//4, h//4))
        annotated[0:h//4, w - w//4 : w] = small

      return annotated, (cx, cy), area

  # if nothing detected
  cv2.putText(annotated, f"No {color_name} detected", (10,30),
              cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
  return annotated, None, 0

def run_color_tracker(color="green", max_frames=200, delay=0.12,
                      min_area=1200, debug=False):
  """Main loop for real-time tracking."""
  print("Starting camera. Allow webcam permission in the browser.")
  time.sleep(0.5)
  for i in range(max_frames):
    try:
      frame = grab_frame(quality=0.7, tries=2)
    except Exception as e:
      print("Capture error:", e)
      return

    annotated, centroid, area = detect_and_track(
        frame, color_name=color, min_area=min_area, debug=debug
    )

    clear_output(wait=True)
    print(f"Frame {i+1}/{max_frames} — Tracking: {color} — area: {area}")
    cv2_imshow(annotated)
    time.sleep(delay)

  print("Finished.")

print("READY ✅  Now run:  run_color_tracker(color='green', max_frames=200, debug=True)")


In [ ]:
run_color_tracker(color='green', max_frames=200, delay=0.12, min_area=1000, debug=True)


In [ ]:
# UPLOAD VIDEO FILE
from google.colab import files
uploaded = files.upload()


In [ ]:
import cv2
from google.colab.patches import cv2_imshow

# use the exact file name you uploaded
cap = cv2.VideoCapture('sample-mp4-file.mp4')

while True:
    ret, frame = cap.read()
    if not ret:
        break

    annotated, centroid, area = detect_and_track(
        frame,
        color_name="green",   # change to "red", "blue", "yellow" based on your video
        min_area=800,
        debug=True
    )

    cv2_imshow(annotated)

cap.release()
